In [1]:
#sqldf() provides a convenient interface of running SQL statement on data frames. 
#Similarly, Python also offers multiple ways to interact between SQL and Pandas DataFrames by 
#leveraging the lightweight SQLite engine.

#import the library SQLite
import sqlite3


In [2]:
# generate some sample numbers
def fib(n):
    a, b = 0, 1

    for _ in range(n):
        yield a
        a, b = b, a + b


In [3]:
#establish a conenction
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()


In [4]:
# check if the DB name exists
with connection:
    cursor.execute('''CREATE TABLE IF NOT EXISTS fib (
                            calculated_value INTEGER)''')
    cursor.executemany('INSERT INTO fib VALUES (?)',
                       [(str(x),) for x in fib(10)])


In [5]:
#run a sample query
cursor.execute('SELECT * FROM fib')
print(cursor.fetchall())

[(0,), (1,), (1,), (2,), (3,), (5,), (8,), (13,), (21,), (34,)]


In [6]:
connection.close()

In [7]:
conn = sqlite3.connect(':memory:')
 
#design a table in the sqlite db
sql = """
    create table sensor (
        sid         integer primary key not null,
        name        text,
        notes       text
    );"""

_ = cur.execute(sql)

NameError: name 'cur' is not defined

In [ ]:
print(sql)

In [ ]:
sql = "insert into sensor(sid, name, notes) values (%d, '%s', '%s');"

In [ ]:
print(sql)

In [ ]:
for (uid, name, notes) in [(201, 'Alpha', 'Sensor for weight'), \
                           (202, 'Beta', 'Sensor for conductivity'),
                           (203, 'Gamma', 'Sensor for surface oxides'),
                           (204, 'Delta', 'Sensor for length'),
                           (205, 'Epsilon', 'Sensor for x-ray'),
                           (206, 'Zeta', 'Color checker 9000'),
                           (207, 'Eta', 'Ultra-violet detector'), ]:
    cur.execute(sql % (uid, name, notes))


In [ ]:
print(sql)

In [ ]:
import numpy as np
from collections import defaultdict
import gc
import time
from pandas import DataFrame
from pandas.util.testing import rands
import random


In [ ]:
N = 10000

In [ ]:
indices = np.array([rands(10) for _ in range(N)], dtype='O')
indices2 = np.array([rands(10) for _ in range(N)], dtype='O')
key = np.tile(indices[:8000], 10)
key2 = np.tile(indices2[:8000], 10)

In [ ]:
indices

In [ ]:
left = DataFrame({'key' : key, 'key2':key2,
                  'value' : np.random.randn(80000)})

In [ ]:
left.head()

In [ ]:
right = DataFrame({'key': indices[2000:], 'key2':indices2[2000:],
                   'value2' : np.random.randn(8000)})


In [ ]:
right.head()

In [ ]:
right2 = right.append(right, ignore_index=True)
right = right2


In [ ]:
right.head()

In [ ]:
right2.head()

In [ ]:
random.shuffle(key2)

In [ ]:
indices2 = indices.copy()

In [ ]:
random.shuffle(indices2)

In [ ]:
# Prepare Database
import sqlite3
create_sql_indexes = False

In [ ]:
conn = sqlite3.connect(':memory:')

In [ ]:
conn.execute('create table left( key varchar(10), key2 varchar(10), value int);')

In [ ]:
conn.execute('create table right( key varchar(10), key2 varchar(10), value2 int);')

In [ ]:
conn.executemany('insert into left values (?, ?, ?)',
                 zip(key, key2, left['value']))

In [ ]:
conn.executemany('insert into right values (?, ?, ?)',
                 zip(right['key'], right['key2'], right['value2']))


In [ ]:
# Create Indices
if create_sql_indexes:
    conn.execute('create index left_ix on left(key, key2)')
    conn.execute('create index right_ix on right(key, key2)')


In [ ]:
join_methods = ['inner', 'left outer', 'left'] # others not supported

In [ ]:
join_methods

In [ ]:
sql_results = DataFrame(index=join_methods, columns=[False])

In [ ]:
sql_results

In [ ]:
niter = 5

In [ ]:
for sort in [False]:
    for join_method in join_methods:
        sql = """select *
        from left
           %s join right
             on left.key=right.key
               and left.key2 = right.key2""" % join_method
        if sort:
            sql = '%s order by key, key2' % sql
        f = lambda: list(conn.execute(sql)) # list fetches results
        g = lambda: conn.execute(sql) # list fetches results
        gc.disable()
        start = time.time()
        for _ in range(niter):
            f()
        elapsed = (time.time() - start) / niter
        gc.enable()
        sql_results[sort][join_method] = elapsed
sql_results.columns = ['sqlite3'] # ['dont_sort', 'sort']
sql_results.index = ['inner', 'outer', 'left']

In [ ]:
sql_results

In [ ]:
#advantages of pysqldf over pandasql are two-fold. 

#1. pysqldf is 2 – 3 times faster than pandasql. 
#2. pysqldf supports new function definitions, which is not available in pandasql. 

#However, the generic python interface to an in-memory SQLite database can be more efficient and 
#flexible than both pysqldf and pandasql

In [ ]:
import sqlalchemy

In [ ]:
sqlalchemy.__version__

#The SQLAlchemy Object Relational Mapper presents a method of associating user-defined Python classes with database tables, and instances of those classes (objects) with rows in their corresponding tables. It includes a system that transparently synchronizes all changes in state between objects and their related rows, called a unit of work, as well as a system for expressing database queries in terms of the user defined classes and their defined relationships between each other.

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [ ]:
#The echo flag is a shortcut to setting up SQLAlchemy logging, which is accomplished via Python’s standard logging module. 

In [ ]:
#The return value of create_engine() is an instance of Engine, and it represents the core interface to the database, 
#adapted through a dialect that handles the details of the database and DBAPI in use. In this case the SQLite dialect will 
#interpret instructions to the Python built-in sqlite3 module.



In [ ]:
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Integer, String

In [ ]:
class User(Base):
...     __tablename__ = 'users'
...
...     id = Column(Integer, primary_key=True)
...     name = Column(String)
...     fullname = Column(String)
...     password = Column(String)
...
...     def __repr__(self):
...        return "<User(name='%s', fullname='%s', password='%s')>" % (
...                             self.name, self.fullname, self.password)

In [ ]:
#With our User class constructed via the Declarative system, we have defined information about our table,
#known as table metadata. The object used by SQLAlchemy to represent this information for a specific table 
#is called the Table object, 
#and here Declarative has made one for us.

In [ ]:
User.__table__ 

#The MetaData is a registry which includes the ability to emit a limited set of schema generation commands to the database. As our SQLite database does not actually have a users table present, we can use MetaData to issue CREATE TABLE statements to the database for all tables that don’t yet exist. 

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [ ]:
ed_user.name

In [ ]:
ed_user.password

In [ ]:
str(ed_user.id)

In [ ]:
from sqlalchemy.orm import sessionmaker

In [ ]:
Session = sessionmaker(bind=engine)

In [ ]:
#In the case where your application does not yet have an Engine when you define your module-level objects

In [ ]:
Session = sessionmaker()

In [ ]:
#when you create your engine with create_engine(), connect it to the Session using configure()

In [ ]:
Session.configure(bind=engine) 

In [ ]:
session = Session()

In [ ]:
#Adding and Updating Objects

In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [ ]:
session.add(ed_user)

In [ ]:
our_user = session.query(User).filter_by(name='ed').first() 

In [ ]:
our_user

In [ ]:
ed_user is our_user

In [ ]:
#We can add more User objects at once using add_all():

In [ ]:
session.add_all([
...     User(name='wendy', fullname='Wendy Williams', password='foobar'),
...     User(name='mary', fullname='Mary Contrary', password='xxg527'),
...     User(name='fred', fullname='Fred Flinstone', password='blah')])

In [ ]:
#the password for Ed isn’t too secure, so lets change it:

In [ ]:
ed_user.password = 'f8s7ccs'

In [ ]:
session.dirty

In [ ]:
session.new

#We tell the Session that we’d like to issue all remaining changes to the database and commit the transaction, which has been in progress throughout. We do this via commit(). 

In [ ]:
session.commit()

In [ ]:
#commit() flushes the remaining changes to the database, and commits the transaction. 

In [ ]:
#Since the Session works within a transaction, we can roll back changes made too.

In [ ]:
ed_user.name = 'Edwardo'

In [ ]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')

In [ ]:
session.add(fake_user)

In [ ]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

In [ ]:
session.rollback()

In [ ]:
ed_user.name

In [ ]:
fake_user in session

In [ ]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

#A Query object is created using the query() method on Session. This function takes a variable number of arguments, which can be any combination of classes and class-instrumented descriptors.

In [ ]:
for instance in session.query(User).order_by(User.id):
...     print(instance.name, instance.fullname)

In [ ]:
for name, fullname in session.query(User.name, User.fullname):
...     print(name, fullname)

In [ ]:
for row in session.query(User, User.name).all():
...    print(row.User, row.name)

In [ ]:
for row in session.query(User.name.label('name_label')).all():
...    print(row.name_label)

In [ ]:
#The name given to a full entity such as User, assuming that multiple entities are present in the call to query(), 
#can be controlled using aliased() :


In [ ]:
from sqlalchemy.orm import aliased

In [ ]:
user_alias = aliased(User, name='user_alias')

In [ ]:
for row in session.query(user_alias, user_alias.name).all():
...    print(row.user_alias)

In [ ]:
#Basic operations with Query include issuing LIMIT and OFFSET, most conveniently using Python array slices and 
#typically in conjunction with ORDER BY:


In [ ]:
for u in session.query(User).order_by(User.id)[1:3]:
...    print(u)

In [ ]:
for name, in session.query(User.name).\
...             filter_by(fullname='Ed Jones'):
...    print(name)

In [ ]:
for name, in session.query(User.name).\
...             filter(User.fullname=='Ed Jones'):
...    print(name)

In [ ]:
for user in session.query(User).\
...          filter(User.name=='ed').\
...          filter(User.fullname=='Ed Jones'):
...    print(user)

In [ ]:
#Here’s a rundown of some of the most common operators used in filter():


In [ ]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)

In [ ]:
query.all() #all() returns a list:

In [ ]:
query.first()

In [ ]:
from sqlalchemy import text

In [ ]:
#Literal strings can be used flexibly with Query, by specifying their use with the text() construct

In [ ]:
for user in session.query(User).\
...             filter(text("id<224")).\
...             order_by(text("id")).all():
...     print(user.name)

In [ ]:
session.query(User).filter(text("id<:value and name=:name")).\
...     params(value=224, name='fred').order_by(User.id).one()

In [ ]:
#To use an entirely string-based statement, a text() construct representing a complete 
#statement can be passed to from_statement(). 

In [ ]:
session.query(User).from_statement(
...                     text("SELECT * FROM users where name=:name")).\
...                     params(name='ed').all()

In [ ]:
stmt = text("SELECT name, id, fullname, password "
...             "FROM users where name=:name")

In [ ]:
stmt = stmt.columns(User.name, User.id, User.fullname, User.password)

In [ ]:
session.query(User).from_statement(stmt).params(name='ed').all()

In [ ]:
#The count() method is used to determine how many rows the SQL statement would return. 

In [ ]:
session.query(User).filter(User.name.like('%ed')).count()

In [ ]:
from sqlalchemy import func

In [ ]:
session.query(func.count(User.name), User.name).group_by(User.name).all()

In [ ]:
#To achieve our simple SELECT count(*) FROM table, we can apply it as:

In [ ]:
session.query(func.count('*')).select_from(User).scalar()

In [ ]:
session.query(func.count(User.id)).scalar()